In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math


In [2]:
from secrets import IEX_CLOUD_API_TOKEN
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 101721167,
 'calculationPrice': 'close',
 'change': -0.53,
 'changePercent': -0.00299,
 'close': 175.16,
 'closeSource': 'flofacii',
 'closeTime': 1685345852681,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 177.66,
 'delayedPriceTime': 1655801568406,
 'extendedChange': -0.42,
 'extendedChangePercent': -0.00245,
 'extendedPrice': 179.5,
 'extendedPriceTime': 1711057795626,
 'high': 184.71,
 'highSource': 'e  m1d eeiidpaelyu5ctrn',
 'highTime': 1687661992637,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 179.407,
 'iexCloseTime': 1671939128453,
 'iexLastUpdated': 1645326453785,
 'iexMarketPercent': 0.012051976490051887,
 'iexOpen': 179.35,
 'iexOpenTime': 1686040109142,
 'iexRealtimePrice': 178.67,
 'iexRealtimeSize': 10,
 'iexVolume': 1318032,
 'lastTradeTime': 1686190149605,
 'latestPrice': 180.59,
 'latestSource': 'Close',
 'latestTime': 'December 9, 2021',
 'latestUpdate': 1695083817505,
 'latestVolume':

In [3]:
stocks = pd.read_csv('starter-files/sp_500_stocks.csv')

In [4]:
describe = "noice"
str = f"python is {describe}"
print(str)

python is noice


In [5]:
print(data['latestPrice'])
print("$",data['marketCap']/1000000000000,"Trillion")

180.59
$ 2.952698477401 Trillion


In [6]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [7]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,180.59,2952698477401,N/A


In [8]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [9]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,157.81,47858710340,N/A
1,AAL,18.16,12038600152,N/A
2,AAP,235.31,15157415417,N/A
3,AAPL,183.23,2977822117865,N/A
4,ABBV,129.44,219506335931,N/A
...,...,...,...,...
500,YUM,137.79,40500496024,N/A
501,ZBH,127.87,26340682286,N/A
502,ZBRA,612.11,33521574096,N/A
503,ZION,63.78,9999504429,N/A


In [10]:
portfolio_size = input("Enter competition portfolio size here: ")

try:
    val = float(portfolio_size)
except ValueError:
    print("Enter the data in numericall form")
    portfolio_size = input("Enter competition portfolio size here: ")

Enter competition portfolio size here: 100000000


In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,157.81,47858710340,1254
1,AAL,18.16,12038600152,10904
2,AAP,235.31,15157415417,841
3,AAPL,183.23,2977822117865,1080
4,ABBV,129.44,219506335931,1529
...,...,...,...,...
500,YUM,137.79,40500496024,1437
501,ZBH,127.87,26340682286,1548
502,ZBRA,612.11,33521574096,323
503,ZION,63.78,9999504429,3104


In [12]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [13]:
background_color = '#ffffff'
font_color = '#000000'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [14]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [15]:
writer.save()